### UCL 2021/22 Data Analysis & Predictions


In [1]:
import pandas as pd
import numpy as np

In [2]:
# Functions used for Clustering
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree
from scipy.spatial.distance import pdist

In [3]:
# Dataframes 
df_attacking = pd.read_csv("data/202122/attacking.csv", index_col = 0)
df_attempts = pd.read_csv("data/202122/attempts.csv", index_col = 0)
df_defending = pd.read_csv("data/202122/defending.csv", index_col = 0)
df_disciplinary = pd.read_csv("data/202122/disciplinary.csv", index_col = 0)
df_distribution = pd.read_csv("data/202122/distributon.csv", index_col = 0)
df_goalkeeping = pd.read_csv("data/202122/goalkeeping.csv", index_col = 0)
df_goals = pd.read_csv("data/202122/goals.csv", index_col = 0)
df_key_stats = pd.read_csv("data/202122/key_stats.csv")

In [4]:
df_fwstats1 = pd.merge(df_attacking, df_attempts, how="outer", on=["player_name", "club", "position"])
df_fwstats2 = pd.merge(df_fwstats1, df_distribution, how="outer", on=["player_name", "club", "position"])
df_fwstats3 = pd.merge(df_fwstats2, df_goals, how="outer", on=["player_name", "club", "position"])
df_fwstats = pd.merge(df_fwstats3, df_key_stats, how="outer", on=["player_name", "club", "position"])
df_fwstats = df_fwstats.loc[df_fwstats['position'] == 'Forward']
df_fwstats = df_fwstats.dropna()
#print(df_fwstats.columns.tolist())
df_fwstats = df_fwstats[['player_name','club','goals_y','assists_y','dribbles','total_attempts','pass_attempted','pass_completed',
'headers','outside_areas','distance_covered']]
df_fwstats

C:\Users\XuanD\AppData\Local\Temp\ipykernel_23008\3834538670.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'match_played_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_fwstats3 = pd.merge(df_fwstats2, df_goals, how="outer", on=["player_name", "club", "position"])


,player_name,club,goals_y,assists_y,dribbles,total_attempts,pass_attempted,pass_completed,headers,outside_areas,distance_covered
1,Vinícius Júnior,Real Madrid,4.0,6.0,83.0,27.0,451.0,377.0,0.0,0.0,133.0
3,Antony,Ajax,2.0,5.0,28.0,26.0,322.0,262.0,0.0,0.0,65.1
8,Mbappé,Paris,6.0,4.0,43.0,27.0,257.0,220.0,0.0,0.0,65.2
9,Gerard Moreno,Villarreal,2.0,4.0,9.0,21.0,163.0,110.0,1.0,0.0,65.7
13,Müller,Bayern,4.0,3.0,2.0,14.0,366.0,283.0,1.0,1.0,98.5
16,Coman,Bayern,2.0,3.0,59.0,20.0,329.0,271.0,0.0,0.0,76.3
19,Zapata,Atalanta,3.0,3.0,10.0,22.0,130.0,92.0,0.0,0.0,50.1
20,Lewandowski,Bayern,13.0,3.0,1.0,31.0,206.0,163.0,1.0,0.0,99.7
30,Forsberg,Leipzig,2.0,2.0,3.0,11.0,164.0,132.0,0.0,1.0,48.2
37,Tadić,Ajax,2.0,2.0,13.0,12.0,303.0,231.0,0.0,0.0,78.9
